In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=338b579c7a32b5740fd916e532d37526c4427e46facb29d053621720f44a4d7a
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=9730ad9064372eac4ec1b8c0ddfb6f46e2482540799708c92fc45c8aebb37433
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 148.4 MB/s eta 0:00:00


In [4]:
from torch.cuda import is_available

from pytorch3d.renderer.mesh.textures import TexturesAtlas
import torch

from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import(
    chamfer_distance,
    mesh_edge_loss,
    mesh_normal_consistency,
    mesh_laplacian_smoothing
)

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    SoftPhongShader,
    MeshRenderer,
    MeshRasterizer,
    RasterizationSettings,
    TexturesAtlas,
    TexturesVertex

)

import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("cannot find gpu")


In [5]:
!pwd

/content


In [6]:
!ls

drive  sample_data


In [7]:
import os
os.chdir('drive/MyDrive/Colab Notebooks')

In [8]:
!ls

 2.1.gif  'a1(1-2).ipynb'   data	        rendering_texture_obj.ipynb
 3_b.gif  'a1(3).ipynb'     deform_form.ipynb


In [9]:
!ls

 2.1.gif  'a1(1-2).ipynb'   data	        rendering_texture_obj.ipynb
 3_b.gif  'a1(3).ipynb'     deform_form.ipynb


In [10]:
DATA_DIR='./data'

obj_file="cow_mesh/cow.obj"

obj_file=os.path.join(DATA_DIR, obj_file)


In [15]:
#rotation of the cow
import imageio

verts, faces, aux= load_obj(obj_file, device=device, load_textures=False )

color=[50,1,100]
textures=torch.ones_like(verts.unsqueeze(0))
textures=textures.to(device)
textures= textures* torch.tensor(color).to(device)


mesh=Meshes(verts=[verts], faces= [faces.verts_idx], textures=TexturesVertex(textures) )
mesh=mesh.to(device)

#settings cameras
nums_view=12
fovs=torch.linspace(5,120,nums_view)
meshes=mesh.extend(nums_view)




results=[]

i=0
for fov in tqdm(fovs):

      distance= 6/(2*(np.tan(np.deg2rad(fov/2))))
      T=[[0,0,distance]]

      cameras=FoVPerspectiveCameras(device=device, fov=fov, T=T)
      raster_settings=RasterizationSettings(
          image_size=256,
          blur_radius=0.0,
          faces_per_pixel=1
      )

      lights=PointLights(device=device, location=[0,0,-3])

      renderer=MeshRenderer(
          MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
          shader=SoftPhongShader(device=device,cameras=cameras)
      )

      image=renderer(meshes[i], cameras=cameras, device=device)
      results.append(image[0, ... , :3].detach().cpu().numpy())
      i+=1


imageio.mimsave("/content/drive/MyDrive/Colab Notebooks/1_b.gif", results, fps=15)

# plt.figure(figsize=(7,7))

# cpu_image=images[0, ... , :3].cpu().numpy()

# plt.imshow(cpu_image)
# plt.axis("off");


  0%|          | 0/12 [00:00<?, ?it/s]